<a href="https://colab.research.google.com/github/LB-TMG/test-google-colab/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [14]:
import warnings
warnings.filterwarnings("ignore")

books = pd.read_csv("/content/BX-Books.csv", sep=';', encoding="latin-1", on_bad_lines="skip")
users = pd.read_csv("/content/BX-Users.csv", sep=';', encoding="latin-1", on_bad_lines="skip")
ratings = pd.read_csv("/content/BX-Book-Ratings.csv", sep=';', encoding="latin-1", on_bad_lines="skip")

books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
books.rename(columns = {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'}, inplace=True)
users.rename(columns = {'User-ID':'user_id', 'Location':'location', 'Age':'age'}, inplace=True)
ratings.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)


# **Data Preparation**

In [15]:
x = ratings['user_id'].value_counts() > 200
y = x[x].index  #user_ids
print(y.shape)
ratings = ratings[ratings['user_id'].isin(y)]

rating_with_books = ratings.merge(books, on='ISBN')
rating_with_books.head()

rating_with_books.to_csv("new_book_dataset.csv")

(1192,)


In [17]:
number_rating = rating_with_books.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns= {'rating':'number_of_ratings'}, inplace=True)
final_rating = rating_with_books.merge(number_rating, on='title')
final_rating.shape
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]
final_rating.drop_duplicates(['user_id','title'], inplace=True)

book_pivot = final_rating.pivot_table(columns='user_id', index='title', values="rating")
book_pivot.fillna(0, inplace=True)

book_pivot.to_csv("book_dataset.csv")

# **Forming the model**

In [18]:
bp = pd.read_csv('/content/book_dataset.csv')
#This uses an unsupervised model
book_sparse = csr_matrix(book_pivot)

model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [20]:
#book_index = book_pivot.index[i]
def get_book_title(book_title):
    book_title = int(bp[bp['title']==book_title].index.values)
    return book_title

get_book_title('A Case of Need')

14

# **Recommendation system**

In [21]:
def recommend_books(book_title):
    distances, suggestions = model.kneighbors(book_pivot.iloc[int(bp[bp['title']==book_title].index.values), :].values.reshape(1, -1))
    for i in range(len(suggestions)):
        return book_pivot.index[suggestions[i]]

In [22]:
recommend_books('Life of Pi')

Index(['Life of Pi', 'Surfacing',
       'Galileo's Daughter: A Historical Memoir of Science, Faith, and Love',
       'Dear Emily', 'The Great Baby Caper'],
      dtype='object', name='title')